<a href="https://colab.research.google.com/github/PythonMachineLearningInKorea/SanghoPark/blob/master/8th_dacon_astronomy_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import math 

from sklearn.ensemble import IsolationForest
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
train = pd.read_csv('/content/drive/My Drive/datrain.csv', index_col=0)
test = pd.read_csv('/content/drive/My Drive/datest.csv', index_col=0)
sample_submission = pd.read_csv('/content/drive/My Drive/sample_submission.csv', index_col=0)
# index_col specifies a certain column as the row index.
sample_submission.describe()

STAR_WHITE_DWARF  STAR_CATY_VAR  ...  SERENDIPITY_MANUAL  SERENDIPITY_BLUE
count           10009.0        10009.0  ...             10009.0           10009.0
mean                0.0            0.0  ...                 0.0               0.0
std                 0.0            0.0  ...                 0.0               0.0
min                 0.0            0.0  ...                 0.0               0.0
25%                 0.0            0.0  ...                 0.0               0.0
50%                 0.0            0.0  ...                 0.0               0.0
75%                 0.0            0.0  ...                 0.0               0.0
max                 0.0            0.0  ...                 0.0               0.0

[8 rows x 19 columns]

In [0]:
# gives numbers to columns

column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [0]:
# Make Fiber tpye
fiber = []
label_dict = {}
# Set eliminates duplicated data. Also, it cannot be sliced. Set is undordered.
for i in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==i]
    # test2 = test[test['fiberID']==i]
 
    # idx2 = test2.index
 
    # train2 is the list(set(train['fiberID']))

    idx1 = train2.index
    #print(set(train.loc[idx1]['type_num']))
    if tuple(set(train.loc[idx1]['type_num'])) not in fiber:
    #tuple is no changeable
        fiber.append(tuple(set(train.loc[idx1]['type_num'])))
    #튜플 집합을 만들어서 fiber 안에 넣어준다. 
    label_dict[i] = list(set(train.loc[idx1]['type_num']))
    #label_dict 안에는 리스트 집합을 만들어 넣어준다.
    
fiber_type = {}
for k in range(len(fiber)):
    fiber_type[fiber[k]] = k
    #위에서 만들어준 fiber의 튜플 집합을 fiber_type에 넣어준다.
fiber_dict = {}  
for j in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==j]
    #test2 = test[test['fiberID']==i]
    idx1 = train2.index
    
    #print(set(train.loc[idx1]['type_num']))
    fiber_dict[j] = fiber_type[tuple(set(train.loc[idx1]['type_num']))]

    #fiberID 별로 fiber_dict에 저장


In [0]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = train.append(test)

In [25]:
all_data['fiberType']

id
0          1
1         34
2          5
3         34
4          1
          ..
209995     5
209996     5
209997    34
209998    34
209999     0
Name: fiberType, Length: 210000, dtype: int64

In [0]:
# ????

all_data['fiberType'] = all_data['fiberID']
Fill_Type = lambda x: fiber_dict[x]
all_data['fiberType'] = all_data['fiberType'].apply(Fill_Type)

In [0]:
# 변수 생성 및 파생변수 만들기
psfMags = ['psfMag_u','psfMag_g','psfMag_r','psfMag_i','psfMag_z']
fiberMags = ['fiberMag_u','fiberMag_g','fiberMag_r','fiberMag_i','fiberMag_z']
petroMags = ['petroMag_u','petroMag_g','petroMag_r','petroMag_i','petroMag_z']
modelMags = ['modelMag_u','modelMag_g','modelMag_r','modelMag_i','modelMag_z']

In [0]:
u_values = ['psfMag_u','fiberMag_u','petroMag_u','modelMag_u']
g_values = ['psfMag_g','fiberMag_g','petroMag_g','modelMag_g']
r_values = ['psfMag_r','fiberMag_r','petroMag_r','modelMag_r']
i_values = ['psfMag_i','fiberMag_i','petroMag_i','modelMag_i']
z_values = ['psfMag_z','fiberMag_z','petroMag_z','modelMag_z']

In [0]:
u_r = ['psfMag_u-r', 'fiberMag_u-r','petroMag_u-r','modelMag_u-r']
u_g = ['psfMag_u-g','fiberMag_u-g',  'petroMag_u-g','modelMag_u-g']
g_r = ['psfMag_g-r', 'fiberMag_g-r',  'petroMag_g-r', 'modelMag_g-r']      
g_z = ['psfMag_g-z','fiberMag_g-z', 'petroMag_g-z','modelMag_g-z'] 
g_i = ['psfMag_g-i','fiberMag_g-i','petroMag_g-i', 'modelMag_g-i']

u_i = ['psfMag_u-i','fiberMag_u-i','petroMag_u-i', 'modelMag_u-i']
u_z = ['psfMag_u-z','fiberMag_u-z','petroMag_u-z', 'modelMag_u-z']
r_i = ['psfMag_r-i','fiberMag_r-i','petroMag_r-i', 'modelMag_r-i']
r_z = ['psfMag_r-z','fiberMag_r-z','petroMag_r-z', 'modelMag_r-z']
i_z = ['psfMag_i-z','fiberMag_i-z','petroMag_i-z', 'modelMag_i-z']

In [0]:
sumur = ['psfMag_u+r', 'fiberMag_u+r','petroMag_u+r','modelMag_u+r']
sumug = ['psfMag_u+g','fiberMag_u+g',  'petroMag_u+g','modelMag_u+g']
sumgr = ['psfMag_g+r', 'fiberMag_g+r',  'petroMag_g+r', 'modelMag_g+r']      
sumgz = ['psfMag_g+z','fiberMag_g+z', 'petroMag_g+z','modelMag_g+z'] 
sumgi = ['psfMag_g+i','fiberMag_g+i','petroMag_g+i', 'modelMag_g+i']

sumui = ['psfMag_u+i','fiberMag_u+i','petroMag_u+i', 'modelMag_u+i']
sumuz = ['psfMag_u+z','fiberMag_u+z','petroMag_u+z', 'modelMag_u+z']
sumri = ['psfMag_r+i','fiberMag_r+i','petroMag_r+i', 'modelMag_r+i']
sumrz = ['psfMag_r+z','fiberMag_r+z','petroMag_r+z', 'modelMag_r+z']
sumiz = ['psfMag_i+z','fiberMag_i+z','petroMag_i+z', 'modelMag_i+z']

In [0]:
cols = all_data.columns.drop(['type','type_num','fiberType','fiberID'])
#drop the columns 'type','type_num','fiberType','fiberID'
pca = PCA(n_components = 2)
# PCA n_components 2
x = all_data[cols].values
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['all_pca1','all_pca2'])
# PCA two columns 
all_data = pd.concat([all_data,principalDf],axis =1)
# All data + PCA components

In [0]:
for i in range(4):
    all_data[str(u_values[i])+'-'+'r'] = all_data[u_values[i]]-all_data[r_values[i]]
    all_data[str(u_values[i])+'-'+'g'] = all_data[u_values[i]]-all_data[g_values[i]]
    all_data[str(g_values[i])+'-'+'r'] = all_data[g_values[i]]-all_data[r_values[i]]
    all_data[str(g_values[i])+'-'+'z'] = all_data[g_values[i]]-all_data[z_values[i]]
    all_data[str(g_values[i])+'-'+'i'] = all_data[g_values[i]]-all_data[i_values[i]]
    
    all_data[str(u_values[i])+'-'+'i'] = all_data[u_values[i]]-all_data[i_values[i]]
    all_data[str(u_values[i])+'-'+'z'] = all_data[u_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'z'] = all_data[r_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'i'] = all_data[r_values[i]]-all_data[i_values[i]]
    all_data[str(i_values[i])+'-'+'z'] = all_data[i_values[i]]-all_data[z_values[i]]
        
        
        
    all_data[str(u_values[i])+'+'+'r'] = all_data[u_values[i]]+all_data[r_values[i]]
    all_data[str(u_values[i])+'+'+'g'] = all_data[u_values[i]]+all_data[g_values[i]]
    all_data[str(g_values[i])+'+'+'r'] = all_data[g_values[i]]+all_data[r_values[i]]
    all_data[str(g_values[i])+'+'+'z'] = all_data[g_values[i]]+all_data[z_values[i]]
    all_data[str(g_values[i])+'+'+'i'] = all_data[g_values[i]]+all_data[i_values[i]]
    
    all_data[str(u_values[i])+'+'+'i'] = all_data[u_values[i]]+all_data[i_values[i]]
    all_data[str(u_values[i])+'+'+'z'] = all_data[u_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'z'] = all_data[r_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'i'] = all_data[r_values[i]]+all_data[i_values[i]]
    all_data[str(i_values[i])+'+'+'z'] = all_data[i_values[i]]+all_data[z_values[i]]

In [0]:
val_groups = [psfMags,fiberMags,petroMags,modelMags,u_values,g_values,r_values,i_values,z_values,u_r,u_g,g_r,g_z,g_i,u_i,u_z,r_i,r_z,i_z,sumur, sumug, sumgr,  sumgz, sumgi, sumui, sumuz, sumri ,sumrz ,sumiz]
group_names = ['psf','fiber','petro','model','u','g','r','i','z','u-r','u-g','g-r','g-z','g-i','u-i','u-z','r-i','r-z','i-z','sumur', 'sumug', 'sumgr',  'sumgz', 'sumgi', 'sumui', 'sumuz', 'sumri' ,'sumrz' ,'sumiz']

In [0]:
# mean and standard pca three components.
pca = PCA(n_components = 3)

for i in range(len(val_groups)):
    
    all_data[str(group_names[i])+'_mean'] = all_data[val_groups[i]].mean(axis =1)
    all_data[str(group_names[i])+'_std'] = all_data[val_groups[i]].std(axis =1)

    x = all_data[val_groups[i]].values
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = [str(group_names[i]) + '_pca1', str(group_names[i]) + '_pca2',str(group_names[i]) + '_pca3'])
    #print(group_names[i])
    #print('explained variance ratio :', pca.explained_variance_ratio_)

    all_data = pd.concat([all_data,principalDf],axis =1)

In [0]:
# Rotation with respect to the center!
def RotateXY(x,y,xc=0,yc=0,angle=0,units="DEGREES"):  
    """Rotate an xy cooordinate about a specified origin  
  
    x,y      xy coordinates  
    xc,yc   center of rotation  
    angle   angle  
    units    "DEGREES" (default) or "RADIANS"  
    """  
    import math  
    x = x - xc  
    y = y - yc  
    if units == "DEGREES":  
        angle = math.radians(angle)  
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + xc  
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + yc  
    return xr, yr  
    # We need the angle as a radian unit. 

In [0]:
import datetime
for i in range(4):
    start = datetime.datetime.now()
    u_g_1 = []
    u_g_2 = []
    g_r_1 = []
    g_r_2 = []
    g_z_1 = []
    g_z_2 = []
    g_i_1 = []
    g_i_2 = []
    r_i_1 = []
    r_i_2 = []
        
    for k in range(len(all_data)):
        
        Rotate = RotateXY(all_data[str(u_values[i])+'-'+'g'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        u_g_1.append(Rotate[0])
        u_g_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'r'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_r_1.append(Rotate[0])
        g_r_2.append(Rotate[1])
        
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'z'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_z_1.append(Rotate[0])
        g_z_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'i'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_i_1.append(Rotate[0])
        g_i_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(r_values[i])+'-'+'i'] .loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        r_i_1.append(Rotate[0])
        r_i_2.append(Rotate[1])   
        
    ug1= pd.DataFrame(data = u_g_1
             , columns = [str(u_values[i])+'-'+'g'+'r451'])
    ug2= pd.DataFrame(data = u_g_2
             , columns = [str(u_values[i])+'-'+'g'+'r452'])
       
    gr1= pd.DataFrame(data = g_r_1
             , columns = [str(g_values[i])+'-'+'r'+'r451'])
    gr2= pd.DataFrame(data = g_r_2
             , columns = [str(g_values[i])+'-'+'r'+'r452'])
        
    gz1= pd.DataFrame(data = g_z_1
             , columns = [str(g_values[i])+'-'+'z'+'r451'])
    gz2= pd.DataFrame(data = g_z_2
             , columns = [str(g_values[i])+'-'+'z'+'r452'])
        
        
    gi1= pd.DataFrame(data = g_i_1
             , columns = [str(g_values[i])+'-'+'i'+'r451'])
    gi2= pd.DataFrame(data = g_i_2
             , columns = [str(g_values[i])+'-'+'i'+'r452'])   
    
    ri1= pd.DataFrame(data = r_i_1
             , columns = [str(r_values[i])+'-'+'i'+'r451'])
    ri2= pd.DataFrame(data = r_i_2
             , columns = [str(r_values[i])+'-'+'i'+'r452'])
             
    
    all_data = pd.concat([all_data,ug1,ug2,gr1,gr2,gz1,gz2,gi1,gi1,ri1,ri2],axis =1)
    runtime = datetime.datetime.now() - start
    break    

In [0]:
# ntrain 할당
train= all_data[:ntrain]
test= all_data[ntrain:]

In [0]:
# N# QSO 34 할당

y = train['type_num']
train = train.drop(columns=['type','type_num'], axis=1)
train1 = train[train['fiberType'] != 34]
train2 = train[train['fiberType'] == 34]
train1_idx = train1.index 
train2_idx = train2.index
y1 = y.loc[train1_idx]

test = test.drop(['type','type_num'], axis=1)
test1 = test[test['fiberType'] != 34]
test2 = test[test['fiberType'] == 34]
test1_idx = test1.index
test2_idx = test2.index

In [0]:
# Hyperparameter tuning : Bayesian optimization
# 20-fold cross validation
# LGBM fitting
from sklearn.model_selection import StratifiedKFold

kfold = 20
kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state = 42)
preds =[]
for train_index, test_index in kf.split(train1, y1):
    x_train, x_val = train1.iloc[train_index], train1.iloc[test_index]
    y_train, y_val = y1.iloc[train_index], y1.iloc[test_index]
    
    clf = lgb.LGBMClassifier(n_estimators = 10000, max_depth=10, learning_rate=0.009, objective='multiclass',num_class = 19
                             ,num_leaves = 1024,random_state=42,metric = 'multi_logloss',boost_from_average = False,
                            feature_fraction =0.21815809919426804,bagging_fraction = 0.8055711236002633,lambda_l1 = 0.0029896841778409566,
                             lambda_l2 = 0.06905300928232105,min_split_gain = 0.05931126989247223,min_child_weight = 10)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)],
            early_stopping_rounds=50, verbose=100)
    
    
    
    # fiberID ==34 인 test_set에 QSO =1로 채워넣음
    ps = pd.DataFrame(data=clf.predict_proba(test), index=test.index)
    sam = sample_submission.copy()
    sam.loc[test1_idx] =ps.loc[test1_idx]
    sam.loc[test2_idx] = 0
    sam.loc[test2_idx,'QSO'] = 1
    

    preds.append(sam)

Training until validation scores don't improve for 50 rounds.
[100]	training's multi_logloss: 1.05226	valid_1's multi_logloss: 1.08366
[200]	training's multi_logloss: 0.612936	valid_1's multi_logloss: 0.663059
[300]	training's multi_logloss: 0.431436	valid_1's multi_logloss: 0.496249
[400]	training's multi_logloss: 0.346715	valid_1's multi_logloss: 0.423345
[500]	training's multi_logloss: 0.302327	valid_1's multi_logloss: 0.389071
[600]	training's multi_logloss: 0.275613	valid_1's multi_logloss: 0.371646
[700]	training's multi_logloss: 0.257378	valid_1's multi_logloss: 0.361985
[800]	training's multi_logloss: 0.243617	valid_1's multi_logloss: 0.356451
[900]	training's multi_logloss: 0.232151	valid_1's multi_logloss: 0.353058
[1000]	training's multi_logloss: 0.222976	valid_1's multi_logloss: 0.350947


In [0]:
# 위에서 구한 20번의 예측값을 더해서 나눠줌.
pred = sum(preds)/20
pred.index = sample_submission.index
pred.columns = sample_submission.columns
pred.to_csv('LGBM_20_Final.csv', index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
